In [1]:
import os.path
import glob
import Input.Input
from Sample import Sample
import cPickle as pickle
import subprocess
import os
import shutil

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# script for converting native instuments stem format mp4 files into wavs using stem2wav
# creates a directory for each filename and places the 5 separated wav files in directory
# requires stempeg ---> 'pip install stempeg'


path = path = os.path.join(os.getcwd(), 'data')
files = [name for name in glob.glob(path + '/*')]
print(files)
for file in files:
    newdir = file[31:-9]
    print(newdir)
    newpath = path + newdir + file[31:]
    print(newpath)
    if not os.path.exists(path + newdir):
        os.makedirs(path + newdir)
        subprocess.call('stem2wav ' + file + ' ' + path + newdir, shell=True)

SyntaxError: invalid syntax (<ipython-input-2-d32c73c38295>, line 6)

In [ ]:

# Mixes together accompaniment of a stem track into acc.wav for unsup_acc use from original
# NI Stems processed with stem2wav and renamed to mix.wav, drums.wav, acc1.wav, acc2.wav, acc3.wav
# SETUP: each track must have a directory containing all five wav files as subdirectories of path
# named as above
# old acc files and the drums and mix files are removed if successful
# requires ffmpeg

path = path = os.path.join(os.getcwd(), 'data')
folders = [name for name in glob.glob(path + '/*')]
print(files)
for folder in folders:
    fullpath = path + folder
    subprocess.call(
        'ffmpeg -i ' + folder + '/acc1.wav -i ' + folder + '/acc2.wav -i ' + folder + \
        '/acc3.wav -filter_complex amix=inputs=3:duration=longest -ac 2 ' + folder + '/acc.wav', shell=True)
    if os.path.exists(folder + '/acc.wav'):
        subprocess.call('rm ' + folder + '/acc1.wav ' + folder + '/acc2.wav ' + folder + '/acc3.wav ' \
                        + folder + '/drums.wav ' + folder + '/mix.wav', shell=True)
    else:
        print('File acc.wav was not created in ' + path)

    
    

In [ ]:

# process nistems for sup
# Mixes together accompaniment of a stem track into drums, mix, and acc from original
# NI Stems processed with stem2wav and renamed to mix.wav, drums.wav, acc1.wav, acc2.wav, acc3.wav
# SETUP: each track must have a directory containing all five wav files as subdirectories of path
# named as above
# old files are removed if successful
# requires ffmpeg

path = path = os.path.join(os.getcwd(), 'data')      #data/musdb18/test'
folders = [name for name in glob.glob(path + '/*')]
print(folders)
for folder in folders:
    print(folder)
    # folder = folder.replace(" ", "\\ ")  # use for spaces in path
    subprocess.call(
        'ffmpeg -i ' + folder + '/acc1.wav -i ' + folder + '/acc2.wav -i ' + folder + \
        '/acc3.wav -filter_complex amix=inputs=3:duration=longest -ac 2 ' + folder + '/acc.wav', shell=True)
    if os.path.exists(folder + '/acc.wav'):
        subprocess.call('rm ' + folder + '/acc1.wav ' + folder + '/acc2.wav ' + folder + '/acc3.wav', shell=True)
    else:
        print('File acc.wav was not created in ' + path)

    

In [24]:
#get list of paths to wav files. musdb18 DB must already be converted using MUSDB docker tool
# data should be in this format:
# ├── test     # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# ├── train    # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# ├── valid    # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# └── train_unsup   # three folders unsup_acc, unsup_drums, unsup_mix. each with numbered
                    # files, ex. acc_1.wav, mix_34.wav, ect in the respective folders

path = os.path.join(os.getcwd(), 'data')
train_paths = [name for name in glob.glob(path + '/train/*')]
test_paths = [name for name in glob.glob(path + '/test/*')]
valid_paths = [name for name in glob.glob(path + '/valid/*')]
unsup_drums_paths = [name for name in glob.glob(path + '/train_unsup/unsup_drums/*')]
unsup_mix_paths = [name for name in glob.glob(path + '/train_unsup/unsup_mix/*')]
unsup_acc_paths = [name for name in glob.glob(path + '/train_unsup/unsup_acc/*')]


In [25]:
dataset = dict()
dataset["train_sup"] = [] # [list of tuples containing sample oblects (mix, acc, drums)]
dataset["train_unsup"] = [] # Initialise unsupervised dataset [[mix_samples], [acc_samples], [drums]]
dataset["valid"] = [] # list of tuples containing sample oblects (mix, acc, drums)
dataset["test"] = [] # list of tuples containing sample oblects (mix, acc, drums)

In [28]:
unsup_mix_paths

['/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_80.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_134.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_5.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_52.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_117.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_124.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_146.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_46.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_115.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_40.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_137.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_163.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_9.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_172.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_157.wav',
 '/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_155.wav',


In [26]:
# create dataset from filename lists using Sample objects
# add_audio() uses librosa which requires python 2.7
for path in train_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['train_sup'].append((mix, acc, drums))

for path in test_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['test'].append((mix, acc, drums))
    
for path in valid_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['valid'].append((mix, acc, drums))

In [29]:
mix = [Sample.from_path(path) for path in unsup_mix_paths]
acc = [Sample.from_path(path) for path in unsup_acc_paths]
drums = [Sample.from_path(path) for path in unsup_drums_paths]
dataset['train_unsup'] = [mix, acc, drums]

    
    


'/Users/dfreezy/DSSGAN'

In [35]:
with open('dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)
print("Created dataset structure")
print('test: ' + str(len(dataset['test'])))
print('valid: ' + str(len(dataset['valid'])))
print('train_sup: ' + str(len(dataset['train_sup'])))
print('train_unsup_mix: ' + str(len(dataset['train_unsup'][0])))
print('train_unsup_acc: ' + str(len(dataset['train_unsup'][1])))
print('train_unsup_drums: ' + str(len(dataset['train_unsup'][2])))

Created dataset structure
test: 50
valid: 50
train_sup: 99
train_unsup_mix: 171
train_unsup_acc: 180
train_unsup_drums: 144


In [33]:
#dataset['train_sup'][0][0].path
dataset['train_unsup'][0][0].path
# dataset['train_unsup'][1][-1].path

'/home/ubuntu/DSSGAN/data/train_unsup/unsup_mix/mix_80.wav'